In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,LabelEncoder
from sklearn.compose import ColumnTransformer

In [6]:
customer_df = pd.read_csv('/content/drive/MyDrive/Zeotap/Customers.csv')
product_df = pd.read_csv('/content/drive/MyDrive/Zeotap/Products.csv')
transaction_df = pd.read_csv('/content/drive/MyDrive/Zeotap/Transactions.csv')


In [10]:
customer_df.head(3)

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07


In [27]:
product_df.head(3)

,ProductID,ProductName,Category,Price,PriceCatagory
0,P001,ActiveWear Biography,Books,169.30,normal
1,P002,ActiveWear Smartwatch,Electronics,346.30,normal
2,P003,ComfortLiving Biography,Books,44.12,cheap


In [26]:
# Calculate quantiles
first_quartile = product_df['Price'].quantile(0.25)
third_quartile = product_df['Price'].quantile(0.75)

# Create the 'priceCatagory' column
def categorize_price(price):
  if price < first_quartile:
    return 'cheap'
  elif price <= third_quartile:
    return 'normal'
  else:
    return 'expensive'

product_df['PriceCatagory'] = product_df['Price'].apply(categorize_price)

In [12]:
transaction_df.head(3)

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68


In [43]:
# prompt: prepare some aggregate columns customerwise from transaction df only

# Aggregate columns
customer_wise_transaction = transaction_df.groupby('CustomerID').agg(
    total_transactions=('TransactionID', 'count'),
    total_revenue=('TotalValue', 'sum'),
    average_transaction_value=('TotalValue', 'mean')
).reset_index()

print(customer_wise_transaction.head())

  CustomerID  total_transactions  total_revenue  average_transaction_value
0      C0001                   5        3354.52                    670.904
1      C0002                   4        1862.74                    465.685
2      C0003                   4        2725.38                    681.345
3      C0004                   8        5354.88                    669.360
4      C0005                   3        2034.24                    678.080


In [14]:
# preparing a map for customer and product
c2p_map = transaction_df[['CustomerID','ProductID']]

In [15]:
# Converting to Datetime
customer_df['SignupDate'] = pd.to_datetime(customer_df['SignupDate'])

In [16]:
customer_df["life"] = (pd.Timestamp.now() - customer_df["SignupDate"]).dt.days

In [101]:
inter_1 = customer_df.merge(c2p_map, on="CustomerID", how="left")
inter_2 = inter_1.merge(product_df, on="ProductID", how="left")
df = customer_wise_transaction.merge(inter_2, on='CustomerID', how='left')

In [102]:
df.head()

,CustomerID,total_transactions,total_revenue,average_transaction_value,CustomerName,Region,SignupDate,life,ProductID,ProductName,Category,Price,PriceCatagory
0,C0001,5,3354.52,670.904,Lawrence Carroll,South America,2022-07-10,932,P054,SoundWave Cookbook,Books,57.30,cheap
1,C0001,5,3354.52,670.904,Lawrence Carroll,South America,2022-07-10,932,P022,HomeSense Wall Art,Home Decor,137.54,cheap
2,C0001,5,3354.52,670.904,Lawrence Carroll,South America,2022-07-10,932,P096,SoundWave Headphones,Electronics,307.47,normal
3,C0001,5,3354.52,670.904,Lawrence Carroll,South America,2022-07-10,932,P083,ActiveWear Smartwatch,Electronics,455.72,expensive
4,C0001,5,3354.52,670.904,Lawrence Carroll,South America,2022-07-10,932,P029,TechPro Headphones,Electronics,433.64,expensive


Feature Engineering

In [120]:
customer_df['nameStartsWith'] = customer_df['CustomerName'].str.lower().str[0]

In [121]:
columns_to_be_dropped = ['CustomerName','SignupDate']

In [123]:
customer_df.drop(columns=columns_to_be_dropped,inplace=True,axis=1)

In [124]:
customer_df.head()

,CustomerID,Region,life,nameStartsWith
0,C0001,South America,932,l
1,C0002,Asia,1079,e
2,C0003,South America,326,m
3,C0004,South America,841,k
4,C0005,Asia,896,l


Scaling and Transform

In [125]:

# Select columns for one-hot encoding and scaling
scaler = MinMaxScaler()
customer_wise_transaction[[ "average_transaction_value", "total_revenue",'total_transactions']] = scaler.fit_transform(
    customer_wise_transaction[["average_transaction_value", "total_revenue",'total_transactions']]
)

In [126]:
customer_wise_transaction.head()

,CustomerID,total_transactions,total_revenue,average_transaction_value
0,C0001,0.4,0.308942,0.474336
1,C0002,0.3,0.168095,0.308940
2,C0003,0.3,0.249541,0.482751
3,C0004,0.7,0.497806,0.473092
4,C0005,0.2,0.184287,0.480120


In [127]:
inter_1 = customer_df.merge(c2p_map, on="CustomerID", how="left")
inter_2 = inter_1.merge(product_df, on="ProductID", how="left")

In [128]:
product_features = inter_2.groupby(['CustomerID', 'Category']).size().unstack(fill_value=0)

        # Normalize purchase frequencies
scaler1 = MinMaxScaler()
product_features_scaled = pd.DataFrame(
scaler.fit_transform(product_features),
index=product_features.index, columns=product_features.columns)

In [131]:
product_features_scaled.head()

Category,Books,Clothing,Electronics,Home Decor
CustomerID,,,,
C0001,0.2,0.0,0.6,0.166667
C0002,0.0,0.4,0.0,0.333333
C0003,0.0,0.2,0.2,0.333333
C0004,0.6,0.0,0.4,0.500000
C0005,0.0,0.0,0.4,0.166667


In [132]:
oheScaler = OneHotEncoder(drop='first')

nominal_features = ['Region', 'nameStartsWith']
ohe_result = oheScaler.fit_transform(customer_df[nominal_features])

# Convert OHE result to DataFrame
ohe_df = pd.DataFrame.sparse.from_spmatrix(ohe_result, columns=oheScaler.get_feature_names_out(nominal_features))

In [150]:
df = pd.concat([customer_df.drop(columns=nominal_features), ohe_df], axis=1)

In [152]:
df.head()

,CustomerID,life,Region_Europe,Region_North America,Region_South America,nameStartsWith_b,nameStartsWith_c,nameStartsWith_d,nameStartsWith_e,nameStartsWith_f,...,nameStartsWith_k,nameStartsWith_l,nameStartsWith_m,nameStartsWith_n,nameStartsWith_p,nameStartsWith_r,nameStartsWith_s,nameStartsWith_t,nameStartsWith_w,nameStartsWith_z
0,C0001,932,0,0,1.0,0,0,0,0,0,...,0,1.0,0,0,0,0,0,0,0,0
1,C0002,1079,0,0,0,0,0,0,1.0,0,...,0,0,0,0,0,0,0,0,0,0
2,C0003,326,0,0,1.0,0,0,0,0,0,...,0,0,1.0,0,0,0,0,0,0,0
3,C0004,841,0,0,1.0,0,0,0,0,0,...,1.0,0,0,0,0,0,0,0,0,0
4,C0005,896,0,0,0,0,0,0,0,0,...,0,1.0,0,0,0,0,0,0,0,0


In [153]:
df = df.merge(product_features_scaled, on="CustomerID", how="left")

In [156]:
df = df.merge(customer_wise_transaction, on="CustomerID", how="left")

In [160]:
from sklearn.metrics.pairwise import cosine_similarity

In [158]:
features = df.drop(columns=["CustomerID"])

In [163]:
features.fillna(0, inplace=True)

In [164]:
similarity_matrix = cosine_similarity(features)

In [167]:
similarity_df = pd.DataFrame(
        similarity_matrix,
        index=features.index,
        columns=customer_df['CustomerID']
    )

In [169]:
similar = similarity_df.loc[0].sort_values(ascending=False)

In [177]:
similar

,0
CustomerID,
C0001,1.000000
C0104,1.000000
C0091,1.000000
C0005,0.999999
C0188,0.999999
...,...
C0105,0.999730
C0066,0.999704
C0117,0.999675


In [194]:
def get_similar_neighbours(index):
    similar_customers = []
    c_id = customer_df['CustomerID'].iloc[index]
    similar = similarity_df.loc[index].sort_values(ascending=False)
    similar = similar[similar.index != c_id]
    similar_customers.append({similar.index[0]:similar.iloc[0]})
    similar_customers.append({similar.index[1]:similar.iloc[1]})
    similar_customers.append({similar.index[2]:similar.iloc[2]})
    return similar_customers

In [203]:
results = {
    'CustomerID':[],
    'Similar_Customers':[]
}

for i in range(20):
  similar_customers = get_similar_neighbours(i)
  c_id = customer_df['CustomerID'].iloc[i]
  results['CustomerID'].append(c_id)
  results['Similar_Customers'].append(similar_customers)

In [ ]:
results

In [205]:
data = pd.DataFrame(results)

In [206]:
data

,CustomerID,Similar_Customers
0,C0001,"[{'C0104': 0.999999695515516}, {'C0091': 0.999..."
1,C0002,"[{'C0162': 0.9999996576963253}, {'C0159': 0.99..."
2,C0003,"[{'C0129': 0.9999995943887455}, {'C0148': 0.99..."
3,C0004,"[{'C0168': 0.9999994729869722}, {'C0188': 0.99..."
4,C0005,"[{'C0001': 0.9999993514385638}, {'C0159': 0.99..."
5,C0006,"[{'C0011': 0.9999970016309214}, {'C0096': 0.99..."
6,C0007,"[{'C0159': 0.999999312090001}, {'C0106': 0.999..."
7,C0008,"[{'C0034': 0.9999962496793716}, {'C0109': 0.99..."
8,C0009,"[{'C0062': 0.999999511118966}, {'C0103': 0.999..."
9,C0010,"[{'C0053': 0.9999997122494227}, {'C0199': 0.99..."


In [207]:
data.to_csv('Lookalike.csv')